In [1]:
import datetime
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
import shapely
from shapely import Point, box
import xagg as xa
import rioxarray

# supress warnings
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

Found esmf.mk at: /home/malasawedah/.conda/envs/test2/lib/esmf.mk


#### Test

In [31]:
import pytest
import pandas as pd
import numpy as np
import xarray as xr
import shutil
import geopandas as gpd
from geopandas import testing as gpdt
from unittest import TestCase
from shapely.geometry import Polygon
from functools import reduce

import pytest
import pandas as pd
import numpy as np
import xarray as xr
import shutil
import geopandas as gpd
from geopandas import testing as gpdt
from unittest import TestCase
from shapely.geometry import Polygon

from xagg.core import (process_weights,create_raster_polygons,get_pixel_overlaps,aggregate,read_wm)
from xagg.wrappers import (pixel_overlaps)


In [32]:
ds = xr.Dataset({'test':(['lon','lat','run'],np.array([[[0,1],[2,3]],[[0,1],[2,3]]])),
				 'lat_bnds':(['lat','bnds'],np.array([[-0.5,0.5],[0.5,1.5]])),
				 'lon_bnds':(['lon','bnds'],np.array([[-0.5,0.5],[0.5,1.5]]))},
				coords={'lat':(['lat'],np.array([0,1])),
						'lon':(['lon'],np.array([0,1])),
						'run':(['run'],np.array([0,1])),
						'bnds':(['bnds'],np.array([0,1]))})

In [33]:
# Create polygon covering multiple pixels
gdf = {'name':['test'],
			'geometry':[Polygon([(0,0),(0,1),(1,1),(1,0),(0,0)])]}
gdf = gpd.GeoDataFrame(gdf,crs="EPSG:4326")

# Get pixel overlaps
wm = pixel_overlaps(ds,gdf)

# Get aggregate
agg = aggregate(ds,wm, stat=['mean'])

creating polygons for each pixel...
calculating overlaps between pixels and output polygons...
success!
aggregating test by mean...
all variables aggregated to polygons!


In [34]:
################# Test to data set ################
ds_out = agg.to_dataset()

# Build reference output dataset
ds_ref = xr.Dataset({'name':(['poly_idx'],np.array(['test']).astype(object)),
                     'test_mean':(['poly_idx','run'],np.array([[1.0,2.0]]))},
                coords={'poly_idx':(['poly_idx'],np.array([0])),
                        'run':(['run'],np.array([0,1]))})

# Assert equal within tolerance, again likely due to very slight 
# variation off from actual 1.0, 2.0 due to crs
xr.testing.assert_allclose(ds_out,ds_ref,atol=0.0001)

In [36]:
################# Test to dataframe ################


df_out = agg.to_dataframe()

# Build reference output dataframe
df_ref = pd.DataFrame({'poly_idx':[0,0],'run':[0,1],'name':['test','test'],'test_mean':[0.9999,1.9999]})
df_ref = df_ref.set_index(['poly_idx','run'])

# Assert equal within tolerance, again likely due to very slight 
# variation off from actual 1.0, 2.0 due to crs
pd.testing.assert_frame_equal(df_out,df_ref,atol=0.0001)